<a href="https://colab.research.google.com/github/DEvauthentic/DEvauthentic/blob/main/ANGULAR_EN_PRODUCTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Étape 1 : Préparer le projet Angular pour PROD

In [ ]:
npm ci

**Vérifier le angular.json**

sourceMap: false pour PROD

optimization: true

buildOptimizer: true

Extrait angular.json :

In [ ]:
"configurations": {
  "production": {
    "fileReplacements": [
      {
        "replace": "src/environments/environment.ts",
        "with": "src/environments/environment.prod.ts"
      }
    ],
    "optimization": true,
    "outputHashing": "all",
    "sourceMap": false,
    "extractCss": true,
    "namedChunks": false,
    "aot": true,
    "extractLicenses": true,
    "vendorChunk": false,
    "buildOptimizer": true
  }
}


Définir le DockerFile (ANGULAR + NGINX)

In [ ]:
# Stage 1: Build Angular app
FROM node:20-alpine AS build

# Définition du repertoire de travail
WORKDIR /app

# On copie le package.jon et on installe les dépendances
COPY package*.json ./
RUN npm ci

# On copie le reste du code se trouvant dans le rep de travail
COPY . .
RUN npm run build -- --configuration production

# Stage 2: Serve with Nginx
FROM nginx:alpine

# Copier build Angular dans Nginx
COPY --from=build /app/dist/nom-de-ton-app /usr/share/nginx/html

# Copier config Nginx custom si nécessaire
COPY nginx.conf /etc/nginx/conf.d/default.conf

# Expose port
EXPOSE 80

CMD ["nginx", "-g", "daemon off;"]


Définir le fichier de configuration de NGINX pour ANGULAR

In [ ]:
server {
  listen 80;

  root /usr/share/nginx/html;
  index index.html;

  location / {
    try_files $uri $uri/ /index.html;
  }

  # Sécurité de base
  add_header X-Frame-Options "SAMEORIGIN";
  add_header X-Content-Type-Options "nosniff";
  add_header X-XSS-Protection "1; mode=block";
}

Construire et tester l'image Docker localement

In [ ]:
docker build -t mon-app-angular .

Lancer le conteneur

In [ ]:
docker run -p 4200:80 mon-app-angular

Étape 5 : Déploiement PROD avec Docker

In [ ]:
docker pull monuser/mon-app-angular:latest
docker stop mon-app || true
docker rm mon-app || true
docker run -d --name mon-app -p 80:80 monuser/mon-app-angular:latest